# Dataloader for contrastive learning with tabular data

In [ ]:
from abc import ABC

import torch
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder

import config

In [ ]:
paths = config.Paths()

### Pytorch Dataset

In [ ]:
class TabularDataset(Dataset, ABC):
    """Tabular Dataset for contrastive learning using a VAE model"""

    def __init__(self, df = None, label_col = 'label'):
        self.label_col = label_col

        super(TabularDataset, self).__init__()
        if df is None:
            self.__get_data()
        else:
            self.data = df

        self.inputs = np.array(self.data.drop(columns=[self.label_col]))
        self.labels = np.array(self.data[self.label_col])

        if type(self.labels[0]) == str:
            self.__encode_labels()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input1 = torch.from_numpy(self.inputs[idx]).float()
        label = torch.from_numpy(np.array(self.labels[idx])).long()

        return input1, label

    def __get_data(self):
        df = pd.read_csv('../../../' + paths.data_dir + 'raw/paldat_complete_clean.csv')

        # drop non numerical columns except for label
        labels = df[self.label_col]
        df = df._get_numeric_data()
        df[self.label_col] = labels

        #normalize columns
        for col in df.columns:
            if col != self.label_col:
                df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

        # drop columns with nan values
        df = df.dropna(axis=1)
        self.data = df

    def __encode_labels(self):
        le = LabelEncoder()
        self.labels = le.fit_transform(self.labels)
        self.label_names = le.classes_

In [ ]:
dataset = TabularDataset(label_col = 'family')

In [ ]:
dataset.__getitem__(0)[0]

In [ ]:
cols = dataset.data.columns

In [ ]:
def get_one_hot_index(cols):
    """
    generates a list of indexes (start, end) for one hot encoded columns based on prefix of column names
    :param cols: list of column names
    :return: list of (start, end) tuples
    """
    one_hot_index = []
    start = 0
    end = 0
    current = cols[0].split('_')[0]
    for i in cols:
        if i.split('_')[0] == current:
            end += 1
        else:
            one_hot_index.append((start, end))
            start = end
            end += 1
            current = i.split('_')[0]

    return one_hot_index

In [ ]:
get_one_hot_index(cols)

In [ ]:
cols

### Triplet Loss Dataset

In [ ]:
class TripletDataset(Dataset, ABC):
    """Tabular Dataset generating triplets for contrastive learning using a VAE model"""

    def __init__(self, df = None, label_col = 'label', setting='train'):
        """
        Initializes the dataset. If no dataframe is given, the data is loaded from the csv file
        and normalized. If a dataframe is given, it is assumed that the data is already normalized
        :param df: dataframe containing data
        :param label_col: name of the column containing the labels
        :param setting: 'train' or 'test'
        """
        self.label_col = label_col

        super(TripletDataset, self).__init__()
        if df is None:
            self.__get_data(setting=setting)
        else:
            self.data = df

        self.inputs = np.array(self.data.drop(columns=[self.label_col]))
        self.labels = np.array(self.data[self.label_col])

        if type(self.labels[0]) == str:
            self.__encode_labels()

        self.one_hot_index = get_one_hot_index(self.data.columns)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        Returns a triplet of data points and their labels
        :param idx: index of anchor point
        :return: anchor, postive, negative, labels
        """
        anchor_idx, postive_idx, negative_idx =  self.__get_triplet(idx)

        anchor = torch.from_numpy(self.inputs[anchor_idx]).float()
        postive = torch.from_numpy(self.inputs[postive_idx]).float()
        negative = torch.from_numpy(self.inputs[negative_idx]).float()

        labels = {'anchor': torch.from_numpy(np.array(self.labels[anchor_idx])).long(),
                  'positive': torch.from_numpy(np.array(self.labels[postive_idx])).long(),
                  'negative': torch.from_numpy(np.array(self.labels[negative_idx])).long()}

        return anchor, postive, negative, labels

    def __get_data(self, setting:str ='train'):
        """
        Loads data from csv file and normalizes it. According to setting, only train or test data is loaded
        :param setting: 'train' or 'test'
        :return: None
        """
        df = pd.read_csv('../../../' + paths.data_dir + 'raw/paldat_complete_clean.csv')

        if setting == 'train':
            df = df[df['setting'] == 'train']
        elif setting == 'test':
            df = df[df['setting'] == 'test']

        df.drop(columns=['setting'], inplace=True)

        # drop non numerical columns except for label
        labels = df[self.label_col]
        df = df._get_numeric_data() # naughty
        df[self.label_col] = labels

        #normalize columns
        for col in df.columns:
            if col != self.label_col:
                df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

        # drop columns with nan values
        df = df.dropna(axis=1)
        self.data = df

    def __encode_labels(self):
        """
        Encodes labels to integers. Saves label names in self.label_names
        :return: None
        """
        le = LabelEncoder()
        self.labels = le.fit_transform(self.labels)
        self.label_names = le.classes_

    def __get_triplet(self, idx):
        """
        Generates a triplet based on target column self.label_col
        :return: triplet
        """
        # get gLabel
        label = self.data.iloc[idx][self.label_col]

        # get anchor and positive from label
        anchor_idx = np.random.choice(self.data[self.data[self.label_col] == label].index)
        positive_idx = np.random.choice(self.data[self.data[self.label_col] == label].index)

        # get negative from different label
        negative_label = random.choice(self.label_names)
        while negative_label == label:
            negative_label = random.choice(self.label_names)

        negative_idx = np.random.choice(self.data[self.data[self.label_col] == negative_label].index)

        return anchor_idx, positive_idx, negative_idx

In [ ]:
dataset = TripletDataset(label_col = 'family')

In [ ]:
dataset.__getitem__(0)

### Dataloader

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)